<a href="https://colab.research.google.com/github/eduion/AIOT/blob/main/HW5/HW5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import tensorflow_datasets as tfds
import pandas as pd

In [2]:
# 加載 CIFAR-10 數據集
train_split, valid_split, test_split = ['train[:90%]', 'train[90%:]', 'test']
train_data, info = tfds.load("cifar10", split=train_split, with_info=True, as_supervised=True)
valid_data = tfds.load("cifar10", split=valid_split, as_supervised=True)
test_data = tfds.load("cifar10", split=test_split, as_supervised=True)

# 資料預處理函數
def preprocess(image, label):
    image = tf.image.resize(image, (32, 32))  # 確保圖像大小一致
    image = tf.cast(image, tf.float32) / 255.0  # 正規化
    label = tf.cast(label, tf.int32)  # 確保標籤為整數
    label = tf.one_hot(label, 10)  # CIFAR-10 有 10 個類別
    return image, label

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.HPVS13_3.0.2/cifar10-train.tfrecord*...:   0%|         …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.HPVS13_3.0.2/cifar10-test.tfrecord*...:   0%|          …

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [3]:
# 批量大小與 AUTOTUNE
batch_size = 64
AUTOTUNE = tf.data.AUTOTUNE
train_num = info.splits['train[:90%]'].num_examples  # 訓練資料數量

# 處理訓練資料
train_data = train_data.shuffle(train_num)\
                       .map(preprocess, num_parallel_calls=AUTOTUNE)\
                       .batch(batch_size)\
                       .prefetch(buffer_size=AUTOTUNE)

valid_data = valid_data.map(preprocess, num_parallel_calls=AUTOTUNE)\
                       .batch(batch_size)\
                       .prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(preprocess, num_parallel_calls=AUTOTUNE)\
                     .batch(batch_size)\
                     .prefetch(buffer_size=AUTOTUNE)

In [4]:
# 加載 VGG16 模型（不包含分類層）
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# 凍結 VGG16 基礎層
vgg16_base.trainable = False

# 建立自定義分類層
model = models.Sequential([
    vgg16_base,  # VGG16 作為特徵提取器
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # CIFAR-10 的 10 類別
])

# 查看模型摘要
model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 1, 1, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,111,242 (57.64 MB)

 Trainable params: 396,554 (1.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [5]:
# 建立儲存目錄與回調函數
model_dir = 'cifar10-logs/models'
os.makedirs(model_dir, exist_ok=True)

log_dir = os.path.join('cifar10-logs', 'model-vgg16')
model_cbk = TensorBoard(log_dir=log_dir)
model_mckp = ModelCheckpoint(
    filepath=os.path.join(model_dir, 'Best-model-vgg16.keras'),
    monitor='val_categorical_accuracy',
    save_best_only=True,
    mode='max'
)

In [7]:
# 編譯模型
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

# 訓練模型
history = model.fit(
    train_data,
    epochs=5,
    validation_data=valid_data,
    callbacks=[model_cbk, model_mckp]
)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 630s 891ms/step - categorical_accuracy: 0.3369 - loss: 1.8627 - val_categorical_accuracy: 0.5000 - val_loss: 1.4787
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 683s 893ms/step - categorical_accuracy: 0.4380 - loss: 1.5864 - val_categorical_accuracy: 0.5326 - val_loss: 1.3645
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 687s 900ms/step - categorical_accuracy: 0.4798 - loss: 1.4806 - val_categorical_accuracy: 0.5416 - val_loss: 1.3085
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 700s 926ms/step - categorical_accuracy: 0.5061 - loss: 1.4148 - val_categorical_accuracy: 0.5554 - val_loss: 1.2629
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 679s 921ms/step - categorical_accuracy: 0.5239 - loss: 1.3634 - val_categorical_accuracy: 0.5626 - val_loss: 1.2402


In [9]:
loss, acc = model.evaluate(test_data)
print('\nModel Accuracy: {}%'.format(acc))

157/157 ━━━━━━━━━━━━━━━━━━━━ 127s 807ms/step - categorical_accuracy: 0.5584 - loss: 1.2484

Model Accuracy: 0.5555999875068665%


In [13]:
loss = [loss]
acc = [acc]

dict = {"test_Loss": loss,
        "test_Accuracy": acc}

print(pd.DataFrame(dict))

              test_Loss         test_Accuracy
0  [1.2608802318572998]  [0.5555999875068665]
